# PRÁCTICA SYSTEMAS DISTRIBUIDOS III
**KAFKA STREAMING + SPARK**

**Descripción:** este notebook va dirigido a la preparación y limpieza de los datos elegidos para la práctica.

**Datos:** se trata de una base de datos con 3 tablas que contienen información sobre los accidentes de Reino Unido desde 2005 hasta 2014. Estos datos pueden obtenerse a través de este enlace: https://www.kaggle.com/benoit72/uk-accidents-10-years-history-with-many-variables

**Miembros del equipo:** Verónica Gómez, Carlos Grande y Pablo Olmos

**GitHub URL:** https://github.com/Akinorev/dataScienceGitR2/tree/master/03_DataScience_III/sd3

## Índice

## Carga de librerías

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import sys, os

# spark libraries
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from operator import add
from operator import sub

## Carga de scripts

In [2]:
import sys
sys.path.append('../02_scripts')
import myKafka

## 2. Contexto y entorno de Spark

In [3]:
packages = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.5"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

print("PYSPARK_SUBMIT_ARGS = ",os.environ["PYSPARK_SUBMIT_ARGS"],"\n")
print("JAVA_HOME = ", os.environ["JAVA_HOME"])

PYSPARK_SUBMIT_ARGS =  --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.5 pyspark-shell 

JAVA_HOME =  /usr/lib/jvm/java-1.8.0-openjdk-amd64


In [4]:
# Create SparkContext object (low level)
sc = SparkContext(appName="PythonStreamingOccupancy")

---

In [16]:
# Every 5 seconds grab the data from the kafka stream. 
ssc = StreamingContext(sc, 5)

## 3. Métodos Auxiliares

### 3.1 Parseador

In [5]:
from datetime import datetime

def parseSample(line):
  s = line.replace('"','').split(",")
  print(s)
  try:
      return [{"rowId": int(s[0]),
               "date": datetime.strptime(s[1], "%Y-%m-%d %H:%M:%S"),
               "temperature": float(s[2]), 
               "humidity": float(s[3]), 
               "light": float(s[4]),
               "co2": float(s[5]), 
               "humidityRatio": float(s[6]),
               "occupancy": bool(s[7][0])}]
  except Exception as err:
      print("Wrong sample format (%s): " % line)
      print(err)
      return []

### 3.2 

In [6]:
# Configura el endpoint para localizar el broker de Kafka
kafkaBrokerIPPort = "127.0.0.1:9092"

# Productor simple (Singleton!)
# from kafka import KafkaProducer
import kafka
class KafkaProducerWrapper(object):
  producer = None
  @staticmethod
  def getProducer(brokerList):
    if KafkaProducerWrapper.producer != None:
      return KafkaProducerWrapper.producer
    else:
      KafkaProducerWrapper.producer = kafka.KafkaProducer(bootstrap_servers=brokerList,
                                                          key_serializer=str.encode,
                                                          value_serializer=str.encode)
      return KafkaProducerWrapper.producer

# Envía resultados a Kafka! (salida)  
def sendResults(itr):
  prod = KafkaProducerWrapper.getProducer([kafkaBrokerIPPort])
  for m in itr:
    prod.send("metrics", key=m[0], value=m[0]+","+str(m[1]))
  prod.flush()

## 3. Lectura de datos en Kafka

In [18]:
# Kafka: Lectura de datos
kafka_port = '0.0.0.0:9092'
kafkaParams = {"metadata.broker.list": kafka_port}
stream = KafkaUtils.createDirectStream(ssc, ["test"], kafkaParams)
stream = stream.map(lambda o: str(o[1]))

---

## 4. Queries

In [ ]:
# Test

### 4.1 Promedio de valores
Calcular el promedio de valores de temperatura, humedad relativa y concentración de CO2 para cada micro-batch, y el promedio de dichos valores desde el arranque de la aplicación.

In [19]:
samples = stream.flatMap(parseSample)
samples = samples.flatMap(lambda x: x.items())

# -----------------------------------
#        PROMEDIO POR BATCH
# -----------------------------------

avgValuesInBatch = samples \
                   .filter(lambda k: k[0] in ("temperature","humidity","co2")) \
                   .mapValues(lambda v: (v,1)) \
                   .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                   .mapValues(lambda v: v[0]/v[1]) \
                   .repartition(1) \
                   .glom() \
                   .map(lambda arr: ("Average values in batch", arr))

avgValuesInBatch.foreachRDD(lambda rdd: rdd.foreachPartition(sendResults))

# -----------------------------------
#        PROMEDIO EN TOTAL
# -----------------------------------

def update_func(new_val, last_val):
    # NEW_VAL FORMAT: [(236, 6)] 
    # in which the tuple indicates the sum of the values in the batch and the count in the batch
    # LAST_VAL FORMAT: (1401,12)
    # in which the tuple indicates the sum of all the past values and the count of all past records
    new_val = new_val[0]
    if last_val != None:
        totalSum = new_val[0] + last_val[0]
        totalCount = new_val[1] + last_val[1]
    else:
        totalSum = new_val[0]
        totalCount = new_val[1]
    return (totalSum, totalCount)
   

avgValuesTotal = samples \
                 .filter(lambda k: k[0] in ("temperature","humidity","co2")) \
                 .mapValues(lambda v: (v,1)) \
                 .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                 .updateStateByKey(update_func) \
                 .mapValues(lambda v: v[0]/v[1]) \
                 .repartition(1) \
                 .glom() \
                 .map(lambda arr: ("Average values in total", arr))
                     

avgValuesTotal.foreachRDD(lambda rdd: rdd.foreachPartition(sendResults))

sc.setCheckpointDir("../01_data/checkpoint/")

## Ejecución de Spark Streaming

In [20]:
ssc.start()
# ssc.awaitTerminationOrTimeout(10) # Espera 10 segs. antes de acabar

In [21]:
ssc.stop(False)